# Implementation of Probabilistic Containment for Normal Distributions

In this notebook, we'll implement the probabilistic definition of containment for band depth proposed by Yohei.

In [9]:
import statdepth
from scipy.integrate import quad
import numpy as np
import pandas as pd 
import scipy.stats as stats
from tqdm import tqdm

from numpy import exp
from scipy.special import erf
from scipy.integrate import quad
from scipy.special import binom

import bigfloat
from bigfloat import BigFloat

Now we'll define our probability, $D(f \mid f_1,...,f_n) = \binom{n}{2}^{-1}\sum_{1 \leq i < j \leq n} Pr(f_i \leq f \leq f_j)$. We'll define our probabilistic band depth function to take two $1\times n$ arrays for the mean/std, respectively. Then each $f_k = N(\mu_k, \sigma^2_k)$ for $k=1,...,n$.  

In [10]:
from statdepth.testing import generate_noisy_univariate

df = generate_noisy_univariate(n=2)

We'll use random samples from $N(0, 1)$ to generate our means and stds for now. 

In [11]:
N = 5
means, stds = np.ones(N), np.random.randint(1, 15, N)

means, stds

(array([1., 1., 1., 1., 1.]), array([ 5, 13, 14,  2,  3]))

We'll first write a function that calculates the depth for a single normal distribution with respect to a set of normals

In [12]:
from statdepth.depth.calculations._helper import _subsequences
from scipy.stats import norm

def f_normal(z: float, parameters):
    mu_i, sigma_i, mu_j, sigma_j, mu, sigma = parameters
    
    return exp(-(mu-z)**2 / (2*sigma))*\
            (1+erf((mu_j-z)/(np.sqrt(2)*np.sqrt(sigma_j))))*\
            (1-erf((mu_i-z)/(np.sqrt(2)*np.sqrt(sigma_i))))


def _normal_depth(means, stds, curr, f):
    n = len(means)
    cols = list(range(n))
    cols.remove(curr)
    S_nj = 0
    subseq = _subsequences(cols, 2)

    for i in tqdm(range(len(subseq))):
        sequence = subseq[i]
        i, j = sequence
        
        parameters = [
            means[i], stds[i], 
            means[j], stds[j], 
            means[curr], stds[curr]
        ]
        
        integral = quad(lambda x: f(x, parameters), -np.inf, np.inf)[0]
        S_nj += integral
        
    return S_nj / binom(n, 2)

As a sanity check, we'll numerically approximate the exact triple integral with Monte Carlo simulation to make sure our simplified integral is correct

In [13]:
from scipy.integrate import tplquad

def normcdf(x, mu, sigma):
    return norm(loc=mu, scale=sigma).cdf(x)

def f_long_norm(z, parameters: list):
    mu_i, sigma_i, mu_j, sigma_j, mu, sigma = parameters
    return (normcdf(z, mu_i, sigma_i)-normcdf(z, mu, sigma)*normcdf(z, mu_j, sigma_j))*\
            norm(loc=mu, scale=sigma).pdf(z)

And finally our function that calculates depth for all functions in the set

In [14]:
def probabilistic_normal_depth(means, stds, f):
    if len(means) != len(stds):
        raise ValueError('Error, len(means) must equal len(stds)')

    depths = []
    for k in tqdm(range(len(means))):
        mc = np.delete(means, [k])
        stdsc = np.delete(stds, [k])
        
        depths.append(    
            _normal_depth(means, stds, k, f)
        )
    
    return pd.DataFrame({
        'means': means,
        'stds': stds,
        'depths': depths
    })

We can now test and plot it to see if things look correct at first glance

In [21]:
df = probabilistic_normal_depth(means, stds, f_long_norm)



100%|██████████| 5/5 [00:33<00:00,  6.72s/it]


Let's take a look at the depths and then visualize them

In [22]:
import plotly.graph_objects as go

def plot_normal_data(N, df):
    d = []

    largest = df.nlargest(N, columns='depths')
    
    for i, (mu, sigma) in enumerate(zip(largest['means'], largest['stds'])):
        x = np.linspace(mu - 3*sigma, mu+3*sigma, 100)
        d.append(
            go.Scatter(
                x=x, 
                y=stats.norm.pdf(x, mu, sigma), 
                mode='lines', 
                line=dict(color='red', width=1)
            )
        )
    
    df = df.drop(largest.index)
    for i, (mu, sigma) in enumerate(zip(df['means'], df['stds'])):
        x = np.linspace(mu - 3*sigma, mu+3*sigma, 100)
        d.append(
            go.Scatter(
                x=x, 
                y=stats.norm.pdf(x, mu, sigma), 
                mode='lines', 
                line=dict(color='blue', width=1)
        ))
    
    go.Figure(
        data=d,
        layout=go.Layout(title=f'Normal Distributions ({N} deepest colored in red)', showlegend=False)
    ).show()

In [24]:
plot_normal_data(1, df)

Let's now implement the probabilistic Poisson distribution. Suppose $f_1,...,f_n$ are independent and follow a Poisson distribution with distinct parameters. Consider $f \sim Poisson(\lambda) \neq f_i \sim Poisson(\lambda_i) \neq f_j \sim Poisson(\lambda_j)$.

In [ ]:
from scipy.special import gamma, gammaincc, factorial
from numpy import exp

def gammainc(a, x):
    return gamma(a) * gammaincc(a, x)

def f_poisson(lambda_i: float, lambda_f: float, lambda_j: float, lim=1000, quiet=False) -> float:
    '''
    Parameters:
    
    lambda_i: 
        mean of f_i (lower function)
    lambda_f: 
        mean of f (function to calculate probabilistic containment)
    lambda_j: 
        mean of f_j (upper function)
    lim=10000: Upper bound of discrete infinite sum
    
    Returns:
    
    float: Probability of containment
    '''
    
    s = 0
    
    for z in range(1, lim):
        num = BigFloat(np.power(lambda_f, z)*(gamma(z)-gammainc(z, lambda_j))*gammainc(1+z, lambda_i))
        denom = BigFloat(factorial(z)*gamma(z)*gamma(1+z))
        
        if num == 0 or denom == 0 and not quiet:
            break
        
        d = BigFloat(num / denom)
        
        s += d
        
    return exp(-lambda_f) * s


Now that we have our definition of containment, let's write the depth calculation function for a given set of data

In [ ]:
def _poisson_depth(means: list, curr: int, to_compute=None):
    n = len(means)
    cols = list(range(n))
    cols.remove(curr)
    S_nj = 0
    subseq = _subsequences(cols, 2)

    for i in tqdm(range(len(subseq))):
        sequence = subseq[i]
        i, j = sequence
        lambda_i, lambda_f, lambda_j = means[i], means[curr], means[j]
        
        S_nj += f_poisson(lambda_i, lambda_f, lambda_j)
        
    return S_nj / binom(n, 2)

def probabilistic_poisson_depth(means: list, to_compute=None):
    depths = []
    for i in tqdm(range(len(means))):
        depths.append(_poisson_depth(means, i, to_compute=to_compute))
    
    return pd.DataFrame({'means': means, 'depths': depths})

probabilistic_poisson_depth(np.random.randint(1, 10, 10))

Since the factorial values quickly approach the maximum value for floats in Python, we'll work on a log scale instead

In [ ]:
from numpy import log 
from scipy.special import loggamma


And now a version for $f_1, ... ,f_n \sim Binomial(n_i, p_i)$ is some vector of parameters for the $i$th observation. 

In [ ]:
from scipy.special import hyp2f1, binom

def f_binom(params: list, lim=1000) -> float:
    n_i, p_i, n, p, n_j, p_j = params
    
    s = 0
    for z in range(1, lim):
        print(f'z is {z}')
        t = (1-p)**(n-z)*p**z*(1-p_i)**n_i*(1-p_j)**(n_j-z)*p_j**z*\
             binom(n, z)*binom(n_j, z)*\
             ((1/(1-p_i))**n_i - (1-p_i)**(-1-z)*p_i**(1+z)*binom(n_i, 1+z)*\
             hyp2f1(1,1-n_i+z, 2+z, p_i/(p_i-1)))*hyp2f1(1, -n_j+z, 1+z, p_j/(p_j-1))
        
        if np.isnan(t):
            print(t)
            break
            
    return s

In [ ]:
f_binom([50,.1,3,.5,4,.3])